In [1]:
import numpy as np
import gc
import pandas as pd
import tensorflow as tf
from keras import layers
from keras import Input, Model
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import progressbar

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

gc.collect()

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

/kaggle/input/cafa-5-ems-2-embeddings-numpy/train_ids.npy
/kaggle/input/cafa-5-ems-2-embeddings-numpy/train_embeddings.npy
/kaggle/input/cafa-5-ems-2-embeddings-numpy/test_ids.npy
/kaggle/input/cafa-5-ems-2-embeddings-numpy/test_embeddings.npy
/kaggle/input/cafa-5-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-5-protein-function-prediction/IA.txt
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta
/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/go-basic.obo


0

In [2]:
train_terms = pd.read_csv("/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv",sep="\t")
print(train_terms.shape)
train_terms.head(5)

(5363863, 3)


,EntryID,term,aspect
0,A0A009IHW8,GO:0008152,BPO
1,A0A009IHW8,GO:0034655,BPO
2,A0A009IHW8,GO:0072523,BPO
3,A0A009IHW8,GO:0044270,BPO
4,A0A009IHW8,GO:0006753,BPO


In [3]:
train_protein_ids = np.load('/kaggle/input/cafa-5-ems-2-embeddings-numpy/train_ids.npy')
train_embeddings = np.load('/kaggle/input/cafa-5-ems-2-embeddings-numpy/train_embeddings.npy')

column_num = train_embeddings.shape[1]
train_data = pd.DataFrame(train_embeddings, columns = ["Column_" + str(i) for i in range(1, column_num+1)])
print(train_data.shape)

(142246, 1280)


In [4]:
terms = train_terms.groupby(['aspect', 'term'])['term'].count().reset_index(name='frequency')
print(terms.groupby('aspect')['term'].nunique())

V = {}
V['BPO'] = 450
V['CCO'] = 150
V['MFO'] = 200

aspect
BPO    21285
CCO     2957
MFO     7224
Name: term, dtype: int64


In [5]:
selected_terms = []
for aspect in ['BPO', 'CCO', 'MFO']:
    selection = terms.loc[(terms.aspect == aspect)]
    selection = selection.nlargest(V[aspect], columns='frequency', keep='first')
    selected_terms += selection.term.to_list()
selected_terms = set(selected_terms)
labels = list(selected_terms)
print(len(labels))

800


In [6]:
bar = progressbar.ProgressBar(maxval=800, widgets=[progressbar.Percentage()])

train_size = train_protein_ids.shape[0]
train_labels = np.zeros((train_size ,800))

print(train_labels.shape, train_protein_ids.shape)

# Convert from numpy to pandas series for better handling
series_train_protein_ids = pd.Series(train_protein_ids)

for i in range(800):
    n_train_terms = train_terms[train_terms['term'] ==  labels[i]]
    
    label_related_proteins = n_train_terms['EntryID'].unique()
    
    # Replace the ith column of train_Y with with that pandas series.
    train_labels[:,i] =  series_train_protein_ids.isin(label_related_proteins).astype(float)
    bar.update(i+1)
    
bar.finish()

labels_df = pd.DataFrame(data = train_labels, columns = labels)
print(labels_df.shape)

(142246, 800) (142246,)


100%                                                                           


(142246, 800)


In [7]:
train_data = train_data.loc[labels_df.index]
print(train_data.shape, labels_df.shape)

(142246, 1280) (142246, 800)


In [8]:
x_train, x_val, y_train, y_val = train_test_split(train_data, labels_df, test_size=0.2, random_state=42)

In [9]:
input_shape = [train_data.shape[1]]
batch_size = 256

inputs = Input(shape=(*input_shape,))

x = layers.BatchNormalization()(inputs)
x = layers.GaussianNoise(stddev=0.1)(x)

x = layers.Dense(units=832, activation='swish')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

x = layers.Dense(units=960, activation='swish')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(units=800,activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=outputs)

loss = tf.keras.losses.BinaryCrossentropy()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=loss,
    metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision()],
)

model.summary()

early_stopping = EarlyStopping(patience=10, min_delta=0.0001, restore_best_weights=True, monitor='val_loss')
callbacks = [early_stopping]

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1280)]            0         
                                                                 
 batch_normalization (BatchN  (None, 1280)             5120      
 ormalization)                                                   
                                                                 
 gaussian_noise (GaussianNoi  (None, 1280)             0         
 se)                                                             
                                                                 
 dense (Dense)               (None, 832)               1065792   
                                                                 
 batch_normalization_1 (Batc  (None, 832)              3328      
 hNormalization)                                                 
                                                             

In [10]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    batch_size=batch_size,
    epochs=40,
    callbacks = callbacks
)

Epoch 1/40
445/445 [==============================] - 26s 55ms/step - loss: 0.1925 - recall: 0.2091 - precision: 0.1414 - val_loss: 0.1063 - val_recall: 0.1511 - val_precision: 0.6390
Epoch 2/40
445/445 [==============================] - 24s 54ms/step - loss: 0.1037 - recall: 0.2055 - precision: 0.5914 - val_loss: 0.0986 - val_recall: 0.2119 - val_precision: 0.6347
Epoch 3/40
445/445 [==============================] - 25s 57ms/step - loss: 0.0994 - recall: 0.2206 - precision: 0.6084 - val_loss: 0.0958 - val_recall: 0.1994 - val_precision: 0.6551
Epoch 4/40
445/445 [==============================] - 25s 55ms/step - loss: 0.0961 - recall: 0.2304 - precision: 0.6224 - val_loss: 0.0945 - val_recall: 0.2142 - val_precision: 0.6550
Epoch 5/40
445/445 [==============================] - 24s 55ms/step - loss: 0.0942 - recall: 0.2406 - precision: 0.6295 - val_loss: 0.0931 - val_recall: 0.2143 - val_precision: 0.6603
Epoch 6/40
445/445 [==============================] - 25s 56ms/step - loss: 0.09

In [11]:
test_df = np.load('/kaggle/input/cafa-5-ems-2-embeddings-numpy/test_embeddings.npy')
test_protein_ids = np.load('/kaggle/input/cafa-5-ems-2-embeddings-numpy/test_ids.npy')

predictions = model.predict(test_df)

4434/4434 [==============================] - 25s 6ms/step


In [12]:
chunk_size = 5_000
chunks = [range(i, min(i + chunk_size, len(predictions))) for i in range(0, len(predictions), chunk_size)]

final_sub = pd.DataFrame()

print(f"processing {len(chunks)} chunks of {chunk_size} predictions each")

for chunk in chunks:
    print(f"processing chunk {chunk}")
    sub = pd.DataFrame(data=predictions[chunk], columns=list(selected_terms), index=test_protein_ids[chunk])
    sub = sub.T.unstack().reset_index(name='prediction')
    final_sub = pd.concat([final_sub, sub])

final_sub.head()

processing 29 chunks of 5000 predictions each
processing chunk range(0, 5000)
processing chunk range(5000, 10000)
processing chunk range(10000, 15000)
processing chunk range(15000, 20000)
processing chunk range(20000, 25000)
processing chunk range(25000, 30000)
processing chunk range(30000, 35000)
processing chunk range(35000, 40000)
processing chunk range(40000, 45000)
processing chunk range(45000, 50000)
processing chunk range(50000, 55000)
processing chunk range(55000, 60000)
processing chunk range(60000, 65000)
processing chunk range(65000, 70000)
processing chunk range(70000, 75000)
processing chunk range(75000, 80000)
processing chunk range(80000, 85000)
processing chunk range(85000, 90000)
processing chunk range(90000, 95000)
processing chunk range(95000, 100000)
processing chunk range(100000, 105000)
processing chunk range(105000, 110000)
processing chunk range(110000, 115000)
processing chunk range(115000, 120000)
processing chunk range(120000, 125000)
processing chunk range(1

,level_0,level_1,prediction
0,Q9ZSA8,GO:0005737,0.132585
1,Q9ZSA8,GO:0002831,0.137478
2,Q9ZSA8,GO:0031669,0.005618
3,Q9ZSA8,GO:0051171,0.000754
4,Q9ZSA8,GO:1901564,0.139190


In [13]:
final_sub.to_csv('submission.tsv', sep='\t', index=False, header=False)

In [14]:
# #hyperparameter searching
# from hyperopt import hp, fmin, tpe, Trials

# input_shape = [train_data.shape[1]]
# batch_size = 256

# space = {
#     #'stddev': hp.uniform('stddev', 0.05, 0.3),
#     'unit1': hp.quniform('unit1', 512, 2048, 64),
#     'unit2': hp.quniform('unit2', 256, 1024, 64),
#     #'dropout1': hp.uniform('dropout1', 0.05, 0.7),
#     #'dropout2': hp.uniform('dropout2', 0.05, 0.7),
#     #'learning_rate': hp.uniform('learning_rate', 0.0001, 0.1)
# }

# def objective(p):
#     inputs = Input(shape=(*input_shape,))
    
#     x = layers.BatchNormalization()(inputs)
#     x = layers.GaussianNoise(stddev=0.1)(x)
    
#     x = layers.Dense(units=p['unit1'], activation='swish')(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Dropout(0.3)(x)
    
#     x = layers.Dense(units=p['unit2'], activation='swish')(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Dropout(0.3)(x)
    
#     outputs = layers.Dense(units=800, activation='sigmoid')(x)
    
#     model = Model(inputs=inputs, outputs=outputs)
    
#     loss = tf.keras.losses.BinaryCrossentropy()
    
#     optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    
#     model.compile(optimizer=optimizer, loss=loss,
#                   metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

#     early_stopping = EarlyStopping(patience=10, min_delta=0.0001, restore_best_weights=True, monitor='val_loss')
#     callbacks = [early_stopping]

#     model.fit(
#         x_train, y_train,
#         validation_data=(x_val, y_val),
#         batch_size=batch_size,
#         epochs=50,
#         callbacks = callbacks
#     )
    
#     loss = model.evaluate(x_val, y_val)[0]
    
#     return loss

# trials = Trials()
# best = fmin(objective, space, algo=tpe.suggest, max_evals=20, trials=trials, verbose=False)

In [15]:
# best_params = trials.best_trial['misc']['vals']
# print("Best hyperparameter values:", best_params)